In [2]:
import os
import librosa
import _pickle as cPickle
from scipy.io.wavfile import read
#from sklearn.mixture import GMM 
#from speakerfeatures import extract_features
from pathlib import Path
import warnings
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
warnings.filterwarnings("ignore")
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,13))# I changed 20 into 13
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    
    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,numcep=13,nfft=1104,appendEnergy = True)# I changed numcep ,nfft values
    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

In [4]:
##featuer extraction and generate file.mfcc
dest = "..\\data\\mfcc"

train_file = "..\\data\\f_list\\wav_list.txt"
file_paths = open(train_file,'r')
features = np.asarray(())
for path in file_paths:
    path = path.strip()
    file_name=os.path.basename(path)
    split_filename=file_name.split('.')
    molecule_name=split_filename[0]
    print(path + ' before')
    audio, sr = librosa.load(path, sr=16000)
    print("Sample rate: {0}Hz".format(sr))
    vector = extract_features(audio,sr)
    
    if features.size == 0:
        features = vector
        features = np.vstack((features, vector))
        print(path + ' middle1')
        picklefile =molecule_name+".mfcc"+".txt"
        #print(features)
        folder=Path(path).parts
        b='AR'
        if folder[3]==b:
            full_path = os.path.join(dest+"\\AR",picklefile)
            print(path + ' after1')
            with open(full_path, 'w') as filehandle:
                for listitem in vector:
                    filehandle.write('%s\n' % listitem)
           
    else:
        features = np.vstack((features, vector))
        print(path + ' middle2')
        picklefile =molecule_name+".mfcc"+".txt"
        #print(features)
        folder=Path(path).parts
        a='EN'
        b='AR'
        if folder[3]==b:
            full_path = os.path.join(dest+"\\AR",picklefile)
            print(path + ' after2')
            with open(full_path, 'w') as filehandle:
                for listitem in vector:
                    filehandle.write('%s\n' % listitem)
        else:
            full_path1 = os.path.join(dest+"\\EN",picklefile)
            print(path + ' after2')
            with open(full_path1, 'w') as filehandle:
                for listitem in vector:
                    filehandle.write('%s\n' % listitem)       

..\data\wav\AR\S001F_A_NA.wav before
Sample rate: 16000Hz
..\data\wav\AR\S001F_A_NA.wav middle1
..\data\wav\AR\S001F_A_NA.wav after1
..\data\wav\AR\S001F_A_SA.wav before
Sample rate: 16000Hz
..\data\wav\AR\S001F_A_SA.wav middle2
..\data\wav\AR\S001F_A_SA.wav after2
..\data\wav\AR\S001F_A_WA.wav before


KeyboardInterrupt: 

In [5]:
## convert time to frame and generate file.seg
dest = "..\\data\\seg"
train_file = "..\\data\\f_list\\lbl_list.txt"
file_paths = open(train_file,'r')
features = np.asarray(())
for path in file_paths:
    path = path.strip()
    file_name=os.path.basename(path)
    split_filename=file_name.split('.')
    molecule_name=split_filename[0]
    picklefile= molecule_name+".seg"+".txt"
    full_path = os.path.join(dest,picklefile)
    print(path)
    #print(molecule_name)
    for pathlbl in path:
        times = [] 
        file = open(path, "r")
        for line in file:
            times.append([float(x) for x in line.strip().split( )[0:2]])
            index=librosa.core.time_to_frames(np.asanyarray(times),sr=sr,hop_length=160,n_fft=None)#hop_length=0.01*16000
            folder=Path(path).parts
            a='EN'
            b='AR'
            if folder[2]==b:
                full_path = os.path.join(dest+"\\AR",picklefile)
                with open(full_path, 'w') as filehandle:
                    for listitem in index:
                        filehandle.write('%s\n' % listitem)
            else:
                full_path1 = os.path.join(dest+"\\EN",picklefile)
                with open(full_path1, 'w') as filehandle:
                     for listitem in index:
                        filehandle.write('%s\n' % listitem)
   # print("times:"+str(times))
    #print("index:"+str(index))
        file.close()

..\data\lbl\AR\S001F_A_NA.lbl.txt
..\data\lbl\AR\S001F_A_SA.lbl.txt
..\data\lbl\AR\S001F_A_WA.lbl.txt
..\data\lbl\AR\S001F_B_NA.lbl.txt
..\data\lbl\AR\S001F_B_SA.lbl.txt
..\data\lbl\AR\S001F_B_WA.lbl.txt
..\data\lbl\AR\S011F_A_NA.lbl.txt
..\data\lbl\AR\S011F_A_SA.lbl.txt
..\data\lbl\AR\S011F_A_WA.lbl.txt
..\data\lbl\AR\S012F_A_NA.lbl.txt
..\data\lbl\AR\S012F_A_SA.lbl.txt
..\data\lbl\AR\S012F_A_WA.lbl.txt
..\data\lbl\AR\S012F_B_NA.lbl.txt
..\data\lbl\AR\S012F_B_SA.lbl.txt
..\data\lbl\AR\S012F_B_WA.lbl.txt
..\data\lbl\AR\S021F_A_NA.lbl.txt
..\data\lbl\AR\S021F_A_SA.lbl.txt
..\data\lbl\AR\S021F_A_WA.lbl.txt
..\data\lbl\AR\S022F_A_NA.lbl.txt
..\data\lbl\AR\S022F_A_SA.lbl.txt
..\data\lbl\AR\S022F_A_WA.lbl.txt
..\data\lbl\AR\S023M_A_NA.lbl.txt
..\data\lbl\AR\S023M_A_WA.lbl.txt
..\data\lbl\AR\S031F_A_NA.lbl.txt
..\data\lbl\AR\S031F_A_SA.lbl.txt
..\data\lbl\AR\S031F_A_WA.lbl.txt
..\data\lbl\AR\S032F_A_NA.lbl.txt
..\data\lbl\AR\S032F_A_SA.lbl.txt
..\data\lbl\AR\S032F_A_WA.lbl.txt
..\data\lbl\AR